In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
aikc_idas_courses_2025_final_competition_path = kagglehub.competition_download('aikc-idas-courses-2025-final-competition')
print('Data source import complete.')

100%|██████████| 14.0M/14.0M [00:00<00:00, 40.4MB/s]

Extracting files...


Data source import complete.


In [ ]:
# be careful, just gpt generated shit - validate it (or write ur own pipe)
# baseline public score: 0.626
# some my ideas for you:
# add scaling !!! (per dataset / per sequence / per batch)
# add more features: magnitudes / angles / angular vel & dist / cumsums
# remove gravity
# 6d quats (https://arxiv.org/pdf/1812.07035)
# use full sgkf validation and compute oof score
# improve model (base idea: lstm / gru / transformer / mamba / cnn1d, maybe try multibranch or combine these models)
# add augs: jitter / time warp / magnitude warp / scaling / window warp / moda / masking
# try filtering: kalman, savgol, firwin, butter
# ensemble
# tta

In [ ]:
import os
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from torch.utils.data import Dataset, DataLoader
from scipy.spatial.transform import Rotation as R
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [ ]:
SEED = 42
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3
WEIGHT_DECAY = 1e-4
MAX_LEN = 500
N_FOLDS = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
set_seed(SEED)

In [ ]:
def preprocess_features(df):
    """
    Convert to WGS and add magnitude
    """

    quats = df[['rot_x', 'rot_y', 'rot_z', 'rot_w']].to_numpy()
    acc_device = df[['acc_x', 'acc_y', 'acc_z']].to_numpy()

    r = R.from_quat(quats)
    acc_world = r.apply(acc_device)

    df['acc_world_x'] = acc_world[:, 0]
    df['acc_world_y'] = acc_world[:, 1]
    df['acc_world_z'] = acc_world[:, 2]

    df['acc_mag'] = np.linalg.norm(acc_device, axis=1)

    return df

In [ ]:
def build_sequences(df, features, is_train=True):
    sequences = []
    grouped = df.groupby('sequence_id', sort=False)

    for seq_id, g in grouped:
        arr = g[features].to_numpy(dtype=np.float32)
        subject = g['subject_id'].values[0]
        rec = {'sequence_id': seq_id, 'subject_id': subject, 'X': arr}
        if is_train:
            rec['y'] = g['gesture'].values[0]

        sequences.append(rec)

    return sequences

In [ ]:
class MotionDataset(Dataset):
    def __init__(self, sequences, le=None, max_len=MAX_LEN):
        self.sequences = sequences
        self.le = le
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        rec = self.sequences[idx]
        x = rec['X']

        if len(x) > self.max_len:
            x = x[:self.max_len]
        else:
            pad = np.zeros((self.max_len - len(x), x.shape[1]), dtype=np.float32)
            x = np.vstack([x, pad])

        if 'y' in rec:
            return torch.tensor(x), torch.tensor(self.le.transform([rec['y']])[0]), rec['sequence_id']

        return torch.tensor(x), torch.tensor(-1), rec['sequence_id']

In [ ]:
class CNN_GRU_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.gru = nn.GRU(128, hidden_dim, num_layers=2,
                          batch_first=True, bidirectional=True, dropout=0.3)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.feature_extractor(x)
        x = x.transpose(1, 2)
        out, _ = self.gru(x)
        pooled = torch.mean(out, dim=1)

        return self.classifier(pooled)

### Pipeline

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df = preprocess_features(train_df)
test_df = preprocess_features(test_df)

In [ ]:
feature_cols = ['acc_world_x', 'acc_world_y', 'acc_world_z', 'acc_mag', 'rot_w', 'rot_x', 'rot_y', 'rot_z']

In [ ]:
# Scaling
scaler = RobustScaler()
train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])
test_df[feature_cols] = scaler.transform(test_df[feature_cols])

In [ ]:
# Data preparation
le = LabelEncoder()
le.fit(train_df['gesture'])
train_seqs = build_sequences(train_df, feature_cols)
test_seqs = build_sequences(test_df, feature_cols, is_train=False)

In [ ]:
# Cross-Validation Loop
subjects = np.array([s['subject_id'] for s in train_seqs])
gkf = GroupKFold(n_splits=N_FOLDS)
test_preds_all = []

In [ ]:
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_seqs, groups=subjects)):
    print(f"\n--- FOLD {fold} ---")

    t_samples = [train_seqs[i] for i in train_idx]
    v_samples = [train_seqs[i] for i in val_idx]

    train_loader = DataLoader(MotionDataset(t_samples, le), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(MotionDataset(v_samples, le), batch_size=BATCH_SIZE)

    model = CNN_GRU_Model(len(feature_cols), 128, len(le.classes_)).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

    best_f1 = 0
    for epoch in range(EPOCHS):
        model.train()
        for x, y, _ in train_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        model.eval()
        preds, targets = [], []
        with torch.no_grad():
            for x, y, _ in val_loader:
                x = x.to(DEVICE)
                logits = model(x)
                preds.extend(logits.argmax(1).cpu().numpy())
                targets.extend(y.numpy())

        f1 = f1_score(targets, preds, average='macro')
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), f'model_fold{fold}.pth')

        scheduler.step()

    print(f"Fold {fold} Best F1: {best_f1:.4f}")

    # Inference for this fold
    model.load_state_dict(torch.load(f'model_fold{fold}.pth'))
    model.eval()
    test_loader = DataLoader(MotionDataset(test_seqs, le), batch_size=BATCH_SIZE)
    fold_preds = []
    with torch.no_grad():
        for x, _, _ in test_loader:
            logits = model(x.to(DEVICE))
            fold_preds.append(torch.softmax(logits, dim=1).cpu().numpy())
    test_preds_all.append(np.concatenate(fold_preds))

In [ ]:
avg_preds = np.mean(test_preds_all, axis=0)
final_labels = avg_preds.argmax(axis=1)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['gesture'] = le.inverse_transform(final_labels)
submission.to_csv('submission.csv', index=False)
print("\nSubmission saved! Total folds ensemble complete.")


--- FOLD 0 ---
Fold 0 Best F1: 0.7743

--- FOLD 1 ---
Fold 1 Best F1: 0.8694

--- FOLD 2 ---
Fold 2 Best F1: 0.7805

--- FOLD 3 ---
Fold 3 Best F1: 0.8068

--- FOLD 4 ---
Fold 4 Best F1: 0.8442

Submission saved! Total folds ensemble complete.
